# Eblan Version

In [15]:
import pandas as pd
import numpy as np
from autogluon.core.metrics import mean_absolute_error
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import seaborn as sns
from torchmetrics.functional import mean_absolute_error

In [16]:
import warnings
warnings.filterwarnings('ignore')

In [20]:
# Читаем данные
train = pd.read_csv('data/raw/train.csv')
sample_submission = pd.read_csv('data/raw/sample_submission.csv')


In [21]:
# Преобразуем дату
train['Date'] = pd.to_datetime(train['Date'].apply(lambda x: x[:-2] + '01'), format='%Y-%m-%d')
train['YearMonth'] = train['Date'].dt.to_period('M')

In [22]:
# Агрегация по месяцам
monthly_data = train.groupby(['Company_ID', 'Product_ID', 'YearMonth'])['Target'].sum().reset_index()

# Преобразование YearMonth обратно в строковый формат
monthly_data['YearMonth'] = monthly_data['YearMonth'].astype(str)

In [24]:
monthly_data

,Company_ID,Product_ID,YearMonth,Target
0,0,1,2019-05,2
1,0,1,2019-06,13
2,0,1,2019-07,1
3,0,1,2019-09,30
4,0,1,2019-11,1
...,...,...,...,...
862331,3,14668,2021-03,1
862332,3,14668,2021-04,1
862333,3,14668,2021-05,1
862334,3,14668,2021-06,1


In [25]:
# Сортировка данных
monthly_data = monthly_data.sort_values(by=['Company_ID', 'Product_ID', 'YearMonth'])

# Создание лаговых признаков (например, за последние 3 месяца)
for lag in range(1, 4):
    monthly_data[f'lag_{lag}'] = monthly_data.groupby(['Company_ID', 'Product_ID'])['Target'].shift(lag)

# Создание признака месяца для учета сезонности
monthly_data['Month'] = pd.to_datetime(monthly_data['YearMonth']).dt.month

# Заполнение пропусков значений лаговых признаков нулями
monthly_data.fillna(0, inplace=True)

monthly_data

,Company_ID,Product_ID,YearMonth,Target,lag_1,lag_2,lag_3,Month
0,0,1,2019-05,2,0.0,0.0,0.0,5
1,0,1,2019-06,13,2.0,0.0,0.0,6
2,0,1,2019-07,1,13.0,2.0,0.0,7
3,0,1,2019-09,30,1.0,13.0,2.0,9
4,0,1,2019-11,1,30.0,1.0,13.0,11
...,...,...,...,...,...,...,...,...
862331,3,14668,2021-03,1,1.0,1.0,1.0,3
862332,3,14668,2021-04,1,1.0,1.0,1.0,4
862333,3,14668,2021-05,1,1.0,1.0,1.0,5
862334,3,14668,2021-06,1,1.0,1.0,1.0,6


In [26]:
# Сортировка данных для корректного сдвига
monthly_data = monthly_data.sort_values(by=['Company_ID', 'Product_ID', 'YearMonth'])

# Создание целевой переменной — Target_next_month
monthly_data['Target_next_month'] = monthly_data.groupby(['Company_ID', 'Product_ID'])['Target'].shift(-1)

# Заполнение пропусков целевой переменной нулями (если нет данных о следующем месяце)
monthly_data['Target_next_month'] = monthly_data['Target_next_month'].fillna(0)

monthly_data.head()

,Company_ID,Product_ID,YearMonth,Target,lag_1,lag_2,lag_3,Month,Target_next_month
0,0,1,2019-05,2,0.0,0.0,0.0,5,13.0
1,0,1,2019-06,13,2.0,0.0,0.0,6,1.0
2,0,1,2019-07,1,13.0,2.0,0.0,7,30.0
3,0,1,2019-09,30,1.0,13.0,2.0,9,1.0
4,0,1,2019-11,1,30.0,1.0,13.0,11,4.0


In [27]:
# Определение последнего месяца в данных
last_month = monthly_data['YearMonth'].max()
last_month_dt = pd.to_datetime(last_month)

# Определение границы для обучающей и валидационной выборки
validation_start = last_month_dt - pd.DateOffset(months=3)

# Создание маски для разделения данных
train_mask = pd.to_datetime(monthly_data['YearMonth']) < validation_start
validation_mask = pd.to_datetime(monthly_data['YearMonth']) >= validation_start

# Разделение данных
train_df = monthly_data[train_mask]
validation_df = monthly_data[validation_mask]

print(f'Обучающая выборка: {train_df.shape}')
print(f'Валидационная выборка: {validation_df.shape}')

Обучающая выборка: (798388, 9)
Валидационная выборка: (63948, 9)


In [47]:
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error

# Определение признаков и целевой переменной
feature_cols = ['lag_1', 'lag_2', 'lag_3', 'Month']
target_col = 'Target_next_month'

X_train = train_df[feature_cols]
y_train = train_df[target_col]

X_val = validation_df[feature_cols]
y_val = validation_df[target_col]

# Создание и обучение модели
model = CatBoostRegressor(
    n_estimators=1000,
    learning_rate=0.05,
    num_leaves=31,
    loss_function='MAE',
    eval_metric='MAE',
    random_state=42
)


In [48]:

model.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    early_stopping_rounds=500,
    verbose=100
)


0:	learn: 43.2701929	test: 36.3055522	best: 36.3055522 (0)	total: 51.3ms	remaining: 51.2s
100:	learn: 25.2694931	test: 31.5800405	best: 31.3343742 (48)	total: 3.44s	remaining: 30.6s
200:	learn: 24.8974685	test: 31.7129370	best: 31.3343742 (48)	total: 6.86s	remaining: 27.3s
300:	learn: 24.6303408	test: 31.7381306	best: 31.3343742 (48)	total: 10.3s	remaining: 23.8s
400:	learn: 24.3667162	test: 31.5485894	best: 31.3343742 (48)	total: 13.7s	remaining: 20.4s
500:	learn: 24.2758353	test: 31.5108589	best: 31.3343742 (48)	total: 17.1s	remaining: 17.1s
Stopped by overfitting detector  (500 iterations wait)

bestTest = 31.3343742
bestIteration = 48

Shrink model to first 49 iterations.


In [46]:
from sklearn.metrics import mean_absolute_error
# Прогноз на валидационной выборке
y_pred = model.predict(X_val)
mae = mean_absolute_error(y_val, y_pred)
print(f'MAE на валидационной выборке: {mae}')

MAE на валидационной выборке: 31.33437519683432


In [42]:
# Определение будущих месяцев
future_months = pd.date_range(start=last_month_dt + pd.offsets.MonthBegin(), periods=3, freq='MS').to_period('M').astype(str)

# Получение последних доступных данных для создания начальных признаков
last_data = monthly_data.groupby(['Company_ID', 'Product_ID']).apply(lambda x: x.sort_values('YearMonth').tail(3)).reset_index(drop=True)

# Создание начального набора данных для прогнозирования
future_predictions = pd.DataFrame()

for month in future_months:
    # Подготовка признаков
    temp = last_data.copy()
    temp['YearMonth'] = month
    temp['lag_1'] = temp['Target']
    temp['lag_2'] = temp.groupby(['Company_ID', 'Product_ID'])['Target'].shift(1).fillna(0)
    temp['lag_3'] = temp.groupby(['Company_ID', 'Product_ID'])['Target'].shift(2).fillna(0)
    temp['Month'] = pd.to_datetime(month).month

    # Выбор признаков
    X_future = temp[feature_cols]

    # Прогноз
    temp['Target_Pred'] = model.predict(X_future, num_iteration=model.best_iteration_)

    # Добавление прогноза в итоговый DataFrame
    temp['Target'] = temp['Target_Pred']
    future_predictions = pd.concat([future_predictions, temp[['Company_ID', 'Product_ID', 'YearMonth', 'Target_Pred']]], ignore_index=True)

    # Обновление last_data для следующего месяца
    last_data['Target'] = temp['Target']

print(future_predictions.head())

TypeError: CatBoostRegressor.predict() got an unexpected keyword argument 'num_iteration'

# O1 Version

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
from catboost import CatBoostRegressor, Pool

# =============== 1. Загрузка данных ===============

train = pd.read_csv('data/raw/train.csv', parse_dates=['Date'])
sample_submission = pd.read_csv('data/raw/sample_submission.csv')


In [2]:
train['Date'] = pd.to_datetime(train['Date'].apply(lambda x: x[:-2] + '01'), format='%Y-%m-%d')


In [3]:

# Для удобства добавим колонки:
# - Month: год-месяц, чтобы группировать данные помесячно
train['Month'] = train['Date'].dt.to_period('M')  # или dt.to_period('M')
train['Month'] = train['Month'].astype(str)       # Преобразуем в строку формата 'YYYY-MM'



In [4]:
# =============== 2. Агрегация данных до уровня (Company_ID, Product_ID, Month) ===============
mode = lambda x: np.nan if x.isna().all() else x.mode()

monthly_agg = (
    train
    .groupby(['Company_ID', 'Product_ID', 'Month'], as_index=False)
    .agg({
        'Target': 'sum',
        # Если нужно, здесь же агрегируем numerical и categorical фичи
        # 'num_0': 'mean', 'num_02': 'sum', ...
        # 'cat_0':mode
        # 'cat_0': lambda x: x.mode()[0], # или другое агрегирование
    })
    .rename(columns={'Target': 'sum_sales'})
)

In [5]:
monthly_agg

,Company_ID,Product_ID,Month,sum_sales
0,0,1,2019-05,2
1,0,1,2019-06,13
2,0,1,2019-07,1
3,0,1,2019-09,30
4,0,1,2019-11,1
...,...,...,...,...
862331,3,14668,2021-03,1
862332,3,14668,2021-04,1
862333,3,14668,2021-05,1
862334,3,14668,2021-06,1


In [7]:
# Если в будущем захотите добавить лаги (sum_sales_{t-1}, sum_sales_{t-2}, ...),
# можно использовать groupby + shift. Но нужно аккуратно работать с индексами (датами):
monthly_agg.sort_values(by=['Company_ID', 'Product_ID', 'Month'], inplace=True)
monthly_agg['Month_date'] = pd.to_datetime(monthly_agg['Month'], format='%Y-%m')

# Для каждого (Company, Product) создадим лаг суммарных продаж
monthly_agg['prev_sum_sales_0'] = monthly_agg.groupby(['Company_ID','Product_ID'])['sum_sales'].shift(0)
monthly_agg['prev_sum_sales_1'] = monthly_agg.groupby(['Company_ID','Product_ID'])['sum_sales'].shift(1)
# monthly_agg['prev_sum_sales_2'] = monthly_agg.groupby(['Company_ID','Product_ID'])['sum_sales'].shift(2)
# monthly_agg['prev_sum_sales_3'] = monthly_agg.groupby(['Company_ID','Product_ID'])['sum_sales'].shift(3)
# monthly_agg['prev_sum_sales_4'] = monthly_agg.groupby(['Company_ID','Product_ID'])['sum_sales'].shift(4)
# monthly_agg['prev_sum_sales_5'] = monthly_agg.groupby(['Company_ID','Product_ID'])['sum_sales'].shift(5)
# monthly_agg['prev_sum_sales_6'] = monthly_agg.groupby(['Company_ID','Product_ID'])['sum_sales'].shift(6)
# monthly_agg['prev_sum_sales_7'] = monthly_agg.groupby(['Company_ID','Product_ID'])['sum_sales'].shift(7)
# monthly_agg['prev_sum_sales_8'] = monthly_agg.groupby(['Company_ID','Product_ID'])['sum_sales'].shift(8)
# monthly_agg['prev_sum_sales_9'] = monthly_agg.groupby(['Company_ID','Product_ID'])['sum_sales'].shift(9)
# monthly_agg['prev_sum_sales_10'] = monthly_agg.groupby(['Company_ID','Product_ID'])['sum_sales'].shift(10)
# monthly_agg['prev_sum_sales_11'] = monthly_agg.groupby(['Company_ID','Product_ID'])['sum_sales'].shift(11)
# monthly_agg['prev_sum_sales_12'] = monthly_agg.groupby(['Company_ID','Product_ID'])['sum_sales'].shift(12)

# Можно добавить 2, 3 и т.д.:
# monthly_agg['prev_sum_sales_2'] = monthly_agg.groupby(['Company_ID','Product_ID'])['sum_sales'].shift(2)
# ...

In [8]:
subset=['prev_sum_sales_0','prev_sum_sales_1']#,'prev_sum_sales_2','prev_sum_sales_3','prev_sum_sales_4','prev_sum_sales_5','prev_sum_sales_6','prev_sum_sales_7','prev_sum_sales_8','prev_sum_sales_9','prev_sum_sales_10','prev_sum_sales_11','prev_sum_sales_12']

In [9]:
# =============== 3. Создаём целевую переменную "продажи в следующем месяце" ===============

# Для каждой строки с месяцем M хотим в таргете иметь sum_sales в M+1
monthly_agg['target'] = monthly_agg.groupby(['Company_ID','Product_ID'])['sum_sales'].shift(-1)

# Убираем строки, где lag или target недоступны
monthly_agg.dropna(subset=subset+['target'], inplace=True)

In [10]:
# =============== 4. Разделение train/valid (пример time-based CV) ===============

# Предположим, мы хотим последние 3 месяца данных использовать как валидацию,
# а ещё раньше — как train. Или делаем K-фолдовую валидацию по месяцам.
# Для простоты тут сделаем один hold-out (последние N месяцев — valid).

# Сортируем по Month_date (по возрастанию времени)
monthly_agg.sort_values(by=['Month_date'], inplace=True)

# Пример: берём валидацию на последние 3 месяца, которые у нас есть в данных
all_months = sorted(monthly_agg['Month_date'].unique())
N_valid_months = 3
valid_threshold = all_months[-N_valid_months]  # первая из последних 3
train_data = monthly_agg[monthly_agg['Month_date'] < valid_threshold]
valid_data = monthly_agg[monthly_agg['Month_date'] >= valid_threshold]

# Если хотим настоящую k-fold по времени, делаем цикл по разным разрезам.
# Ниже — для простоты делаем 1 разрез.

In [11]:
# =============== 5. Подготовка признаков для модели ===============

# Допустим, в модель пойдут следующие фичи:
# - 'prev_sum_sales_1' (предыдущий месяц продаж)
# - (в будущем можно добавить ещё лаги)
# - Company_ID, Product_ID как категориальные, если это целесообразно
# - и другие categorical: cat_01 … cat_14, агрегированные помесячно (не показано в примере)

feature_cols = subset
cat_cols = ['Company_ID', 'Product_ID']
# Если добавили ещё лаги: feature_cols += ['prev_sum_sales_2', 'prev_sum_sales_3', ...]

X_train = train_data[feature_cols + cat_cols]
y_train = train_data['target']

X_valid = valid_data[feature_cols + cat_cols]
y_valid = valid_data['target']

# Укажем, какие столбцы считать категориальными для CatBoost
# (CatBoost умеет работать с ними напрямую, выучивая таргет-кодирование)
cat_features_indices = [X_train.columns.get_loc(c) for c in cat_cols]  # индексы категориальных фич


In [12]:
X_train

,prev_sum_sales_0,prev_sum_sales_1,Company_ID,Product_ID
411523,2,3.0,2,5107
812254,47,75.0,3,12299
502783,3,2.0,2,11923
502839,9,9.0,2,11925
692048,3,13.0,3,6919
...,...,...,...,...
456089,1,1.0,2,8492
79187,2,1.0,0,5964
377945,1,1.0,2,2536
377466,200,20.0,2,2500


In [13]:
# =============== 6. Обучение CatBoost ===============
model = CatBoostRegressor(
    iterations=1000,
    learning_rate=0.03,
    depth=6,
    random_seed=42,
    cat_features=cat_features_indices,
    eval_metric='MAE',
    verbose=100
)

model.fit(
    X_train,
    y_train,
    eval_set=(X_valid, y_valid),
    early_stopping_rounds=100
)


0:	learn: 67.6988700	test: 76.8085787	best: 76.8085787 (0)	total: 132ms	remaining: 2m 11s
100:	learn: 27.0560873	test: 33.9956120	best: 33.9956120 (100)	total: 4.99s	remaining: 44.4s
200:	learn: 24.4441151	test: 31.4419742	best: 31.4419742 (200)	total: 8.9s	remaining: 35.4s
300:	learn: 23.9749610	test: 31.0231205	best: 31.0230528 (298)	total: 12.7s	remaining: 29.6s
400:	learn: 23.6379484	test: 30.7340051	best: 30.7340051 (400)	total: 17.2s	remaining: 25.6s
500:	learn: 23.3911203	test: 30.5464982	best: 30.5464982 (500)	total: 21.3s	remaining: 21.2s
600:	learn: 23.0963548	test: 30.3300862	best: 30.3300862 (600)	total: 25.6s	remaining: 17s
700:	learn: 22.8902439	test: 30.1623270	best: 30.1623270 (700)	total: 29.8s	remaining: 12.7s
800:	learn: 22.7491334	test: 30.0992794	best: 30.0967622 (796)	total: 34.1s	remaining: 8.47s
900:	learn: 22.6052125	test: 30.0299962	best: 30.0271117 (895)	total: 38.3s	remaining: 4.21s
999:	learn: 22.4916813	test: 29.9305512	best: 29.9284484 (997)	total: 42.1s	

In [14]:
# Оценим качество:
from sklearn.metrics import mean_absolute_error
y_pred_valid = model.predict(X_valid)
mae_valid = mean_absolute_error(y_valid, y_pred_valid)
print("MAE на валидации:", mae_valid)

MAE на валидации: 29.928449430464543


In [13]:

# =============== 7. Финальное обучение на всём train (при желании) и прогноз ===============

# (а) Можно переобучить модель на всём датасете без разделения или оставить так,
#     если хотим финально предсказывать.
# Тут пример как «добрать» valid в train, если хотим максимизировать данные.
full_train = monthly_agg.dropna(subset=feature_cols + ['target'])
X_full = full_train[feature_cols + cat_cols]
y_full = full_train['target']

final_model = CatBoostRegressor(
    iterations=1000,
    learning_rate=0.03,
    depth=6,
    random_seed=42,
    cat_features=cat_features_indices,
    eval_metric='MAE',
    verbose=100
)

final_model.fit(
    X_full,
    y_full
)

0:	learn: 68.4827483	total: 55ms	remaining: 54.9s
100:	learn: 29.6775287	total: 4.82s	remaining: 42.9s
200:	learn: 27.1074566	total: 8.4s	remaining: 33.4s
300:	learn: 26.1120510	total: 12.2s	remaining: 28.4s
400:	learn: 25.0063799	total: 16.2s	remaining: 24.2s
500:	learn: 24.2986328	total: 20s	remaining: 19.9s
600:	learn: 23.9180223	total: 23.6s	remaining: 15.7s
700:	learn: 23.7056397	total: 27s	remaining: 11.5s
800:	learn: 23.5560506	total: 30.5s	remaining: 7.58s
900:	learn: 23.4592937	total: 34.1s	remaining: 3.75s
999:	learn: 23.3876396	total: 37.6s	remaining: 0us


In [14]:







# =============== 8. Предикт на нужные 3 месяца вперёд ===============

# Предположим, мы хотим предсказать продажи на 2023-12, 2024-01, 2024-02 (пример).
# Но у нас в sample_submission.csv уже есть (Id, Target), где Id = Company_Product_Month.

# Разберём sample_submission:
sample_submission['Company_ID'] = sample_submission['Id'].apply(lambda x: x.split('_')[0]).astype(int)
sample_submission['Product_ID'] = sample_submission['Id'].apply(lambda x: x.split('_')[1]).astype(int)
sample_submission['Month_str'] = sample_submission['Id'].apply(lambda x: x.split('_')[2])

# Подразумевается, что Month_str – это месяц, например, '2023-12', '2024-01', '2024-02'
# (как именно в данных – нужно уточнить формат).
# Для прогноза нам нужно сформировать фичи (prev_sum_sales_1, prev_sum_sales_2, ...)
# на предшествующие месяцы.

# В простейшем случае: prev_sum_sales_1 = sum_sales за (Month-1).
# Поэтому нам надо соединить sample_submission с monthly_agg, подтянув к каждому
# (Company_ID, Product_ID, месяц) знание о сумме покупок в прошлом месяце.

# Сначала убедимся, что у monthly_agg есть строки за нужные месяцы (Month_date) = (Month-1)
# Допустим, в sample_submission для месяца '2023-12' нужно взять '2023-11' из monthly_agg.

# Сконвертируем Month_str в datetime, чтобы вычесть 1 месяц
sample_submission['Month_date'] = pd.to_datetime(sample_submission['Month_str'])
sample_submission['Month_date_lag1'] = sample_submission['Month_date'] - pd.offsets.MonthBegin(1)

# Преобразуем их обратно в формат 'YYYY-MM'
sample_submission['Month_str_lag1'] = sample_submission['Month_date_lag1'].dt.to_period('M').astype(str)

# Теперь джойним (Company_ID, Product_ID, Month_str_lag1) с monthly_agg,
# чтобы найти sum_sales за предыдущий месяц (назовём её prev_sum_sales_1).
tmp_agg = monthly_agg[['Company_ID','Product_ID','Month','sum_sales']].copy()
tmp_agg.rename(columns={'Month':'Month_str', 'sum_sales':'prev_sum_sales_1'}, inplace=True)

prediction_df = pd.merge(
    sample_submission,
    tmp_agg,
    left_on=['Company_ID','Product_ID','Month_str_lag1'],
    right_on=['Company_ID','Product_ID','Month_str'],
    how='left',
    suffixes=('', '_agg')
)

# Заполним пропуски нулями, если нет данных:
prediction_df['prev_sum_sales_1'] = prediction_df['prev_sum_sales_1'].fillna(0)

# Формируем нужные признаки для финальной модели:
X_pred = prediction_df[['prev_sum_sales_1', 'Company_ID', 'Product_ID']]
# учтём порядок столбцов:
X_pred = X_pred[feature_cols + cat_cols]

# Предсказание:
prediction_df['Target'] = final_model.predict(X_pred).clip(0, None)  # clip на всякий случай

# Если модель даёт отрицательные прогнозы, их можно обрезать до 0
# Но лучше разбираться, почему негативный прогноз появился.

# Формируем выход
submission = prediction_df[['Id', 'Target']].copy()
submission.to_csv('data/submissions/my_catboost_solution.csv', index=False)

print("Готово! Файл my_catboost_solution.csv сформирован.")

Готово! Файл my_catboost_solution.csv сформирован.
